In [ ]:
import SimpleITK as sitk
import numpy as np
import tensorflow as tf
import math

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def plot_histogram(data_col, title, xlabel, ylabel, x_range=None):
    """
    Plot histogram showing the distribution of patient ages.
    
    Parameters:
    data (Series): Pandas Series containing patient ages.
    """
    plt.figure(figsize=(10, 6))
    sns.histplot(data_col, bins=20, kde=True, color='skyblue')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if x_range:
        plt.xlim(x_range)
    plt.show();

In [ ]:
def forresnet(image_sample):
    """
    Preprocess the image for ResNet-50 model
    
    """
    # Assuming images[0] is a SimpleITK image
    image_sitk = image_sample


    # # Selecting a single slice (e.g., the first slice)
    # single_slice = image_sitk[0, :, :]

    # Convert SimpleITK image to numpy array
    image_array = sitk.GetArrayFromImage(image_sitk)  #single_slice
    middle_slice_index = math.floor(image_array.shape[0] / 2)
    image_array = image_array[middle_slice_index, :, :]
    print("Original shape:", image_array.shape)  # (23, 384)

    # Reshape to add an extra dimension
    image_array = np.expand_dims(image_array, axis=-1)
    print("Shape after adding dimension:", image_array.shape)  # (23, 384, 1)

    # Resize the image to (224, 224)
    image_array = tf.image.resize(image_array, (224, 224))
    print("Resized shape:", image_array.shape)  # (224, 224, 1)

    # Remove the extra dimension
    image_array = tf.squeeze(image_array, axis=-1)
    print("Shape after squeezing:", image_array.shape)  # (224, 224)
    
    # Convert to grayscale NOT NEEDED BECAUSE IT IS ALREADY IN GRAYSCALE
    # image_array = tf.image.rgb_to_grayscale(image_array)
    # print("Shape after converting to grayscale:", image_array.shape)  # (224, 224, 1)

    # Stack to create a 3-channel image
    image_array = tf.stack([image_array] * 3, axis=-1)
    print("Shape after stacking:", image_array.shape)  # (224, 224, 3)

    # Normalize the image
    image_array = image_array / 255.0
    print("Shape after normalization:", image_array.shape)

    # # Add batch dimension
    # image_array = tf.expand_dims(image_array, axis=0)
    # print("Final shape with batch dimension:", image_array.shape)

    return image_array